In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import eval_extraction
import openai
import extract
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()

# Load the data

In [2]:
df = pd.read_pickle("../data/data_clean.pkl")[
    [
        "id",
        "paper___url",
        "paper___raw_text",
        "paper___year",
        'categorization___chief_complaint',
        'categorization___specialty',
        'categorization___purpose',
        'categorization___system',
        'categorization___disease',
    ]
]

In [3]:
df.tail(n=300)

,id,paper___url,paper___raw_text,paper___year,categorization___chief_complaint,categorization___specialty,categorization___purpose,categorization___system,categorization___disease
390,2076,https://www.ncbi.nlm.nih.gov/pubmed/12520701,"See discussions, stats, and author profiles fo...",2003,"[Hematuria, Weight Loss/Gain]","[Hematology and Oncology, Urology]",[Calculation],[Urinary],[Cancer]
391,1985,http://www.ncbi.nlm.nih.gov/pubmed/12924748,Full Terms & Conditions of access and use can ...,2003,"[Anxious, Diarrhea, Headache, Vomiting]","[Emergency Medicine, Family Practice, Geriatri...",[Diagnosis],[Psychiatric],"[Drug/Alcohol Use, Ingestion/Overdose, Substan..."
392,780,https://www.ncbi.nlm.nih.gov/pubmed/17659153,46\nCJEM • JCMU\nJanuary • janvier 2003; 5 (1)...,2003,[Injury/Trauma],"[Critical Care, Emergency Medicine, Hospitalis...","[Prognosis, Treatment]","[Immunologic, Infectious]",[HIV/AIDS]
393,324,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,RESPIRATORY INFECTION\nDefining community acqu...,2003,"[Cough, Fever, Shortness of Breath]","[Critical Care, Emergency Medicine, Family Pra...",[Prognosis],"[Cardiac, Infectious, Respiratory]","[COVID-19, Pneumonia]"
394,4012,https://www.ncbi.nlm.nih.gov/pubmed/12765673,Clinical Nutrition (2003) 22(3): 321–336\nr 20...,2002,"[Fatigue, Weight Loss/Gain]","[Critical Care, Gastroenterology, Clinical Nut...",[Prognosis],[Gastrointestinal],"[Malnutrition, Sepsis]"
...,...,...,...,...,...,...,...,...,...
685,10187,NaN,None,-1,"[Weight Loss/Gain, Anxious, Chest Pain, Shortn...","[Hematology and Oncology, Pulmonology, Interna...","[Calculation, Prognosis]",[Respiratory],"[Cancer, Lung Cancer, Drug/Alcohol Use]"
686,3401,NaN,None,-1,"[Cough, Fever, Sore Throat]","[Allergy and Immunology, Family Practice, Geri...",[Treatment],[Immunologic],[Public Health]
687,2048,NaN,None,-1,"[Hematuria, Weight Loss/Gain]","[Hematology and Oncology, Pathology, Urology]","[Diagnosis, Prognosis]","[Oncologic, Urinary]",[Cancer]
688,2040,NaN,None,-1,"[AMS, Anxious, Bleeding, Rash, Weight Loss/Gain]","[Allergy and Immunology, Critical Care, Dermat...",[Calculation],"[Allergic, Any/All, Dermatologic, Endocrine an...","[Autoimmune Disorders, Ingestion/Overdose]"
